In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import unicodedata

***USER INFORMATION***

---




In [43]:
user_info = pd.read_csv('/content/drive/MyDrive/AI Project/user_info.csv')

In [44]:
user_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424170 entries, 0 to 424169
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         424170 non-null  int64  
 1   age             328803 non-null  float64
 2   sex             418652 non-null  object 
 3   phone           402962 non-null  object 
 4   job             402962 non-null  object 
 5   carrier         402962 non-null  object 
 6   marital_status  423813 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 22.7+ MB


In [45]:
user_info.head(5)

,user_id,age,sex,phone,job,carrier,marital_status
0,376517,44.0,gentle,******6654,Thực tập sinh giáo dục,other,Married
1,234512,39.0,lady,NaN,NaN,vietnamobile,Married
2,344532,39.0,she,******6296,DevOps Engineer,other,cưới
3,186135,37.0,she,******1502,Thực tập sinh giáo dục,vietnamobile,cưới
4,30230,38.0,female,******4966,DevOps Engineer,viettel,Married


In [46]:
# Check missing value
user_info.isna().sum()

,0
user_id,0
age,95367
sex,5518
phone,21208
job,21208
carrier,21208
marital_status,357


In [47]:
user_info.columns

Index(['user_id', 'age', 'sex', 'phone', 'job', 'carrier', 'marital_status'], dtype='object')

In [48]:
print(f"Age min: {user_info['age'].min()}")
print(f"Age max: {user_info['age'].max()}")

Age min: -1.0
Age max: 999.0


In [49]:
# Đếm số người có tuổi > 100
over_80 = (user_info['age'] > 80).sum()

# Đếm số người có tuổi < 12
under_16 = (user_info['age'] < 16).sum()

# Tổng số người không hợp lệ
total_invalid = over_80 + under_16

print(f"Số người có tuổi > 80: {over_80}")
print(f"Số người có tuổi < 16: {under_16}")
print(f"Tổng số người có tuổi không hợp lệ: {total_invalid}")


Số người có tuổi > 80: 708
Số người có tuổi < 16: 721
Tổng số người có tuổi không hợp lệ: 1429


In [50]:
# Lọc ra các giá trị outliers
user_info_1 = user_info[(user_info['age'].between(16,80)  | user_info['age'].isna())].reset_index(drop=True).copy()

# In ra số lượng và các giá trị outliers
print(f"Số lượng outliers: {len(user_info) - len(user_info_1)}")
print(f"Check outlers at age: {(user_info_1['age'] <16).sum() + (user_info_1['age'] >80).sum()}")
print(user_info_1.info())

Số lượng outliers: 1429
Check outlers at age: 0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         422741 non-null  int64  
 1   age             327374 non-null  float64
 2   sex             417235 non-null  object 
 3   phone           401598 non-null  object 
 4   job             401601 non-null  object 
 5   carrier         401610 non-null  object 
 6   marital_status  422384 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 22.6+ MB
None


In [51]:
user_info_1.isna().sum()

,0
user_id,0
age,95367
sex,5506
phone,21143
job,21140
carrier,21131
marital_status,357


In [52]:
user_info_1.sex.unique()

array(['gentle', 'lady', 'she', 'female', 'girl', 'male', 'M', nan, 'boy',
       'woman', 'F', 'he', 'men', 'unknown', 'other', 'mаle', 'mеn', 'hе',
       'malе', 'lаdy', 'wοman', 'fеmale', 'femаle', 'bοy', 'shе', 'gіrl',
       'womаn', 'gеntle', 'սnknown', 'femalе', 'gentlе', 'othеr', 'οther',
       'unknοwn', 'nսll', 'femаlе', 'fеmalе', 'wοmаn', 'mаlе', 'fеmаle',
       'gеntlе'], dtype=object)

In [53]:
user_info_2 = user_info_1.copy()

# Remove leading and trailing whitespace
user_info_2['sex'] = user_info_2['sex'].astype(str).str.strip()

# Normalize Unicode to the canonical decomposed form (NFD).
user_info_2['sex'] = user_info_2['sex'].apply(lambda x: unicodedata.normalize('NFKD', x))

# Remove non-ASCII characters.
user_info_2['sex'] = user_info_2['sex'].apply(lambda x: x.encode('ascii', 'ignore').decode())

# Convert to lowercase.
user_info_2['sex'] = user_info_2['sex'].str.lower()


# Female group

user_info_2.loc[user_info_2['sex'].isin(['lady', 'she', 'female', 'girl','woman', 'f','ldy', 'wman', 'fmale', 'femle', 'sh', 'grl','womn','femal', 'feml', 'fmal', 'wmn', 'fmle']), 'sex'] = 'F'
user_info_2.loc[user_info_2['sex'].isin(['gentle','male', 'm','boy','he', 'men', 'mle', 'mn','h', 'mal','by','gntle', 'gentl','ml','gntl']), 'sex'] = 'M'
user_info_2.loc[user_info_2['sex'].isin(['other','othr', 'ther']), 'sex'] = 'other'
user_info_2.loc[user_info_2['sex'].isin(['unknown','nknown', 'unknwn', 'null', 'nan', 'nll']), 'sex'] = np.nan

In [54]:
user_info_2.sex.unique()

array(['M', 'F', nan, 'other'], dtype=object)

In [55]:
user_info_2.marital_status.unique()

array(['Married', 'cưới', 'Single', 'ly hôn', 'Divorced', nan, 'độc thân',
       'Unmarried'], dtype=object)

In [56]:
user_info_3 = user_info_2.copy()
user_info_3.loc[user_info_3['marital_status'].isin(['Married', 'cưới',]), 'marital_status'] = 'Married'
user_info_3.loc[user_info_3['marital_status'].isin(['Single', 'Unmarried','độc thân']), 'marital_status'] = 'Single'
user_info_3.loc[user_info_3['marital_status'].isin(['ly hôn','Divorced']), 'marital_status'] = 'Divorced'
user_info_3.loc[user_info_3['marital_status'].isin(['nan']), 'marital_status'] = np.nan

In [57]:
user_info_3.marital_status.unique()

array(['Married', 'Single', 'Divorced', nan], dtype=object)

In [58]:
user_info_3.job.unique()

array(['Thực tập sinh giáo dục', nan, 'DevOps Engineer',
       'Kiến trúc sư phần mềm', 'UX/UI Designer',
       'Trưởng phòng kinh doanh', 'Lập trình viên',
       'Nhà phân tích dữ liệu', 'Y tá', 'Quản lý sản xuất',
       'Kỹ sư giám sát', 'Digital Marketer', 'Nghệ sĩ', 'Thực Tập',
       'Điều Dưỡng', 'Kiến trúc sư', 'Nông dân', 'Chuyên viên nhân sự',
       'Nhân viên kinh doanh', 'Nhân viên hành chính', 'Nhà báo',
       'Nhân viên nhà hàng', 'Bác sĩ', 'Chuyên viên tư vấn tuyển sinh',
       'Kỹ sư nông nghiệp', 'Thợ xây', 'Giảng viên',
       'Chuyên viên bảo mật', 'Kỹ sư xây dựng', 'Kỹ sư phần mềm',
       'Nhà thiết kế đồ họa', 'Project Manager', 'Nhà tư vấn tài chính',
       'Nhân viên khách sạn', 'Kiểm toán', 'Thư ký pháp lý',
       'Công nhân sản xuất', 'Nhân viên lễ tân', 'Chuyên viên tín dụng',
       'Giáo viên', 'Sales Representative', 'Tester', 'Kỹ sư sản xuất',
       'Biên dịch viên', 'Luật sư', 'Chuyên viên bán hàng trực tuyến',
       'Chuyên viên phân tích thị 

In [59]:
user_info_4 = user_info_3.copy()
job_nrm = (user_info_4['job'].astype('string').str.strip()
           .str.normalize('NFD').str.encode('ascii','ignore').str.decode('utf-8')
           .str.lower().str.replace(r'\s+', ' ', regex=True))
job_nrm

,job
0,thuc tap sinh giao duc
1,<NA>
2,devops engineer
3,thuc tap sinh giao duc
4,devops engineer
...,...
422736,kien truc su phan mem
422737,tu van khach hang
422738,truong phong kinh doanh
422739,duoc si


In [60]:
user_info_4['job_industry'] = "Others"

user_info_4.loc[job_nrm.isin([
    'devops engineer','lap trinh vien','ky su phan mem','software engineer',
    'tester','qa','nha phan tich du lieu','data analyst','chuyen vien bao mat'
]), 'job_industry'] = 'IT/Software/Data'

user_info_4.loc[job_nrm.isin([
    'ux/ui designer','ux designer','ui designer','nha thiet ke do hoa','designer','nghe si'
]), 'job_industry'] = 'Design/Creative'

user_info_4.loc[job_nrm.isin([
    'marketing manager','digital marketer','sales representative',
    'nhan vien ban hang','nhan vien kinh doanh','chuyen vien ban hang truc tuyen'
]), 'job_industry'] = 'Sales/Marketing'

user_info_4.loc[job_nrm.isin([
    'ke toan','kiem toan','nhan vien ngan hang','chuyen vien tin dung','nha tu van tai chinh'
]), 'job_industry'] = 'Finance/Banking'

user_info_4.loc[job_nrm.isin(['luat su','thu ky phap ly']), 'job_industry'] = 'Legal'
user_info_4.loc[job_nrm.isin(['bac si','y ta','dieu duong','duoc si']), 'job_industry'] = 'Healthcare'
user_info_4.loc[job_nrm.isin(['giao vien','giang vien','thuc tap sinh giao duc']), 'job_industry'] = 'Education/Training'
user_info_4.loc[job_nrm.isin(['ky su xay dung','ky su giam sat','kien truc su','tho xay','ky su san xuat']), 'job_industry'] = 'Engineering/Construction'
user_info_4.loc[job_nrm.isin(['quan ly san xuat','cong nhan san xuat']), 'job_industry'] = 'Manufacturing/Operations'
user_info_4.loc[job_nrm.isin(['nhan vien nha hang','nhan vien khach san','nhan vien le tan']), 'job_industry'] = 'Hospitality/Service'
user_info_4.loc[job_nrm.isin(['nhan vien hanh chinh','thu ky','chuyen vien nhan su']), 'job_industry'] = 'Admin/HR'
user_info_4.loc[job_nrm.isin(['nong dan','ky su nong nghiep']), 'job_industry'] = 'Agriculture'
user_info_4.loc[job_nrm.isin(['nha bao','bien dich vien']), 'job_industry'] = 'Media/Comms'


In [61]:
user_info_4.job_industry.unique()

array(['Education/Training', 'Others', 'IT/Software/Data',
       'Design/Creative', 'Healthcare', 'Manufacturing/Operations',
       'Engineering/Construction', 'Sales/Marketing', 'Agriculture',
       'Admin/HR', 'Media/Comms', 'Hospitality/Service',
       'Finance/Banking', 'Legal'], dtype=object)

In [62]:
user_info_4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         422741 non-null  int64  
 1   age             327374 non-null  float64
 2   sex             411553 non-null  object 
 3   phone           401598 non-null  object 
 4   job             401601 non-null  object 
 5   carrier         401610 non-null  object 
 6   marital_status  422384 non-null  object 
 7   job_industry    422741 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 25.8+ MB


In [63]:
user_info_4.head()

,user_id,age,sex,phone,job,carrier,marital_status,job_industry
0,376517,44.0,M,******6654,Thực tập sinh giáo dục,other,Married,Education/Training
1,234512,39.0,F,NaN,NaN,vietnamobile,Married,Others
2,344532,39.0,F,******6296,DevOps Engineer,other,Married,IT/Software/Data
3,186135,37.0,F,******1502,Thực tập sinh giáo dục,vietnamobile,Married,Education/Training
4,30230,38.0,F,******4966,DevOps Engineer,viettel,Married,IT/Software/Data


In [64]:
user_info_4.columns

Index(['user_id', 'age', 'sex', 'phone', 'job', 'carrier', 'marital_status',
       'job_industry'],
      dtype='object')

In [65]:
user_info_5 = user_info_4[['user_id', 'age', 'sex','marital_status', 'job_industry']].copy()
user_info_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         422741 non-null  int64  
 1   age             327374 non-null  float64
 2   sex             411553 non-null  object 
 3   marital_status  422384 non-null  object 
 4   job_industry    422741 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 16.1+ MB


In [66]:
missing_df = pd.DataFrame({
    'missing_count': user_info_5.isna().sum(),
    'missing_%': (user_info_5.isna().sum() / len(user_info_5) * 100).round(2)
})
missing_df.sort_values('missing_%', ascending=False)


,missing_count,missing_%
age,95367,22.56
sex,11188,2.65
marital_status,357,0.08
user_id,0,0.00
job_industry,0,0.00


In [67]:
# 1) Fill age with random integers (16–40)
age_na_mask = user_info_5['age'].isna()
user_info_5.loc[age_na_mask, 'age'] = np.random.randint(16, 41, age_na_mask.sum())

# 2) Fill sex with random values ("F", "M", "other")
sex_na_mask = user_info_5['sex'].isna()
user_info_5.loc[sex_na_mask, 'sex'] = np.random.choice(['F', 'M', 'other'], sex_na_mask.sum())

# 3) Fill marital_status with mode
marital_mode = user_info_5['marital_status'].mode()[0]
user_info_5['marital_status'] = user_info_5['marital_status'].fillna(marital_mode)


In [68]:
user_info_5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         422741 non-null  int64  
 1   age             422741 non-null  float64
 2   sex             422741 non-null  object 
 3   marital_status  422741 non-null  object 
 4   job_industry    422741 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 16.1+ MB


In [69]:
user_info_5.head(5)

,user_id,age,sex,marital_status,job_industry
0,376517,44.0,M,Married,Education/Training
1,234512,39.0,F,Married,Others
2,344532,39.0,F,Married,IT/Software/Data
3,186135,37.0,F,Married,Education/Training
4,30230,38.0,F,Married,IT/Software/Data


In [70]:
user_info_5.to_csv('/content/drive/MyDrive/AI Project/user_info_final.csv', index=False)

In [71]:
test = pd.read_csv('/content/drive/MyDrive/AI Project/user_info_final.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         422741 non-null  int64  
 1   age             422741 non-null  float64
 2   sex             422741 non-null  object 
 3   marital_status  422741 non-null  object 
 4   job_industry    422741 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 16.1+ MB


***USER LOG***

---



In [5]:
user_log = pd.read_csv('/content/drive/MyDrive/AI Project/user_log.csv')

***TRAIN AND TEST DATA***

---



In [7]:
train_data = pd.read_csv('/content/drive/MyDrive/AI Project/train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/AI Project/test.csv')

In [ ]:
print(f"Train size: {len(train_data)}")
print(f"Test size: {len(test_data)}")

Train size: 233782
Test size: 52695


In [8]:
# Check merchant_id that only align with a single user_id
merchant_user_counts = (
    user_log.groupby('merchant_id')['user_id']
    .nunique()
)

valid_merchants = merchant_user_counts[merchant_user_counts == 1].index
valid_merchants


Index([], dtype='int64', name='merchant_id')

In [9]:
# Check user_id that only align with a single merchant_id

user_merchant_counts = (
    user_log.groupby('user_id')['merchant_id']
    .nunique()
)

valid_users = user_merchant_counts[user_merchant_counts == 1].index

# Filter
df_single_user_merchant = user_log[user_log['user_id'].isin(valid_users)].copy()
df_single_user_merchant.info()


<class 'pandas.core.frame.DataFrame'>
Index: 5973 entries, 316671 to 54924990
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   user_id      5973 non-null   int64  
 1   item_id      5973 non-null   int64  
 2   cat_id       5973 non-null   int64  
 3   brand_id     5948 non-null   float64
 4   merchant_id  5973 non-null   int64  
 5   action       5973 non-null   object 
 6   datetime     5973 non-null   object 
dtypes: float64(1), int64(4), object(2)
memory usage: 373.3+ KB


In [10]:
# From the earlier filtered df that contains only single-merchant users
user_merchant_map = df_single_user_merchant.groupby('user_id')['merchant_id'].first()
user_merchant_map


,merchant_id
user_id,
3724,4426
10582,1618
12250,1407
15998,2482
16521,2537
...,...
412918,3022
415324,4818
416698,1272


In [11]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 233782 entries, 0 to 233781
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      228192 non-null  float64
 1   merchant_id  228284 non-null  float64
 2   label        233782 non-null  float64
dtypes: float64(3)
memory usage: 5.4 MB


In [12]:
train_data.isna().sum()

,0
user_id,5590
merchant_id,5498
label,0


In [13]:
# fill merchant_id when it is missing (NaN) and user_id exists in the mapping

train_data_1 = train_data.copy()

mask = train_data_1['merchant_id'].isna() & train_data_1['user_id'].isin(user_merchant_map)

train_data_1.loc[mask, 'merchant_id'] = train_data_1.loc[mask, 'user_id'].map(user_merchant_map)

train_data_1.isna().sum()


,0
user_id,5590
merchant_id,5498
label,0


In [14]:
train_data_1.label.unique()

array([ 0.,  1., -1.])

In [15]:
train_data_2 = train_data_1[train_data_1['label'] != -1].copy()
train_data_2.label.unique()

array([0., 1.])

In [16]:
train_data_3 = train_data_2.copy()
train_data_3 = train_data_3.dropna().reset_index(drop=True)
train_data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219557 entries, 0 to 219556
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      219557 non-null  float64
 1   merchant_id  219557 non-null  float64
 2   label        219557 non-null  float64
dtypes: float64(3)
memory usage: 5.0 MB


In [17]:
train_data_3.to_csv("/content/drive/MyDrive/AI Project/train_data_final.csv", index=False)

In [ ]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52695 entries, 0 to 52694
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   user_id      52695 non-null  int64
 1   merchant_id  52695 non-null  int64
 2   label        52695 non-null  int64
dtypes: int64(3)
memory usage: 1.2 MB


In [ ]:
test_data.label.unique()

array([0, 1])

In [ ]:
test_data.isna().sum()

,0
user_id,0
merchant_id,0
label,0


In [18]:
# Merge train and test data
full_data = pd.concat([train_data_3, test_data], ignore_index=True)
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272252 entries, 0 to 272251
Data columns (total 3 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   user_id      272252 non-null  float64
 1   merchant_id  272252 non-null  float64
 2   label        272252 non-null  float64
dtypes: float64(3)
memory usage: 6.2 MB


In [31]:
user_info_final = pd.read_csv("/content/drive/MyDrive/AI Project/user_info_final.csv")
user_info_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422741 entries, 0 to 422740
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Unnamed: 0      422741 non-null  int64  
 1   user_id         422741 non-null  int64  
 2   age             422741 non-null  float64
 3   sex             422741 non-null  object 
 4   marital_status  422741 non-null  object 
 5   job_industry    422741 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 19.4+ MB


In [33]:
user_info_final.columns

Index(['Unnamed: 0', 'user_id', 'age', 'sex', 'marital_status',
       'job_industry'],
      dtype='object')

In [34]:
final_full_data = full_data.copy()
final_full_data = pd.merge(final_full_data, user_info_final[['user_id', 'age', 'sex', 'marital_status','job_industry']], on='user_id', how='left')
final_full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272252 entries, 0 to 272251
Data columns (total 7 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   user_id         272252 non-null  float64
 1   merchant_id     272252 non-null  float64
 2   label           272252 non-null  float64
 3   age             267623 non-null  float64
 4   sex             267623 non-null  object 
 5   marital_status  267623 non-null  object 
 6   job_industry    267623 non-null  object 
dtypes: float64(4), object(3)
memory usage: 14.5+ MB


In [20]:
user_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54925330 entries, 0 to 54925329
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   brand_id     float64
 4   merchant_id  int64  
 5   action       object 
 6   datetime     object 
dtypes: float64(1), int64(4), object(2)
memory usage: 2.9+ GB


In [21]:
# only keep the records that have user_id and merchant_id exist in the train-test dataset

user_log_filtered = user_log[(user_log['user_id'].isin(full_data['user_id'])) & user_log['merchant_id'].isin(full_data['merchant_id'])].reset_index(drop=True)

# user_log_filtered = (
#     user_log
#     .merge(
#         full_data[['user_id', 'merchant_id']].drop_duplicates(),
#         on=['user_id', 'merchant_id'],
#         how='inner'
#     )
#     .reset_index(drop=True)
# )


user_log_filtered.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11443682 entries, 0 to 11443681
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   user_id      int64  
 1   item_id      int64  
 2   cat_id       int64  
 3   brand_id     float64
 4   merchant_id  int64  
 5   action       object 
 6   datetime     object 
dtypes: float64(1), int64(4), object(2)
memory usage: 611.2+ MB


In [ ]:
user_log_filtered.isna().sum()

,0
user_id,0
item_id,0
cat_id,0
brand_id,21108
merchant_id,0
action,0
datetime,0


In [22]:
# Change datatype of datetime
user_log_1 = user_log_filtered[['user_id', 'item_id', 'merchant_id', 'action', 'datetime']].copy()
user_log_1['datetime'] = pd.to_datetime(user_log_1['datetime'])

user_log_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11443682 entries, 0 to 11443681
Data columns (total 5 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   merchant_id  int64         
 3   action       object        
 4   datetime     datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 436.5+ MB


In [ ]:
user_log_1.isna().sum()

,0
user_id,0
item_id,0
merchant_id,0
action,0
datetime,0


In [23]:
print(f"Start time: {user_log_1['datetime'].min()}")
print(f"End time: {user_log_1['datetime'].max()}")

Start time: 2024-05-11 00:00:00
End time: 2024-11-12 00:00:00


In [24]:
# Adding is_sale_day feature

# Define Shopee double-day mega sale days for time range above
sale_days = [
    "2024-05-15", "2024-05-25",
    "2024-06-06", "2024-06-15", "2024-06-25",
    "2024-07-07", "2024-07-15", "2024-07-25",
    "2024-08-08", "2024-08-15", "2024-08-25",
    "2024-09-02", "2024-09-09", "2024-09-15", "2024-09-25",
    "2024-10-10", "2024-10-15", "2024-10-20", "2024-10-25",
    "2024-11-11", "2024-11-15", "2024-11-25",
]

sale_days = pd.to_datetime(sale_days)

# Flag whether each row is a sale-day interaction
user_log_1['is_sale_day'] = user_log_1['datetime'].dt.normalize().isin(sale_days)

# Adding date and month feature
user_log_1['date'] = user_log_1['datetime'].dt.normalize()
user_log_1['month'] = user_log_1['datetime'].dt.to_period('M')

user_log_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11443682 entries, 0 to 11443681
Data columns (total 8 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   merchant_id  int64         
 3   action       object        
 4   datetime     datetime64[ns]
 5   is_sale_day  bool          
 6   date         datetime64[ns]
 7   month        period[M]     
dtypes: bool(1), datetime64[ns](2), int64(3), object(1), period[M](1)
memory usage: 622.1+ MB


In [25]:
user_log_1.head()

,user_id,item_id,merchant_id,action,datetime,is_sale_day,date,month
0,234512,240182,3018,purchase,2024-11-11,True,2024-11-11,2024-11
1,234512,137298,3271,purchase,2024-11-11,True,2024-11-11,2024-11
2,234512,137298,3271,click,2024-11-11,True,2024-11-11,2024-11
3,234512,719701,4655,click,2024-11-11,True,2024-11-11,2024-11
4,234512,137298,3271,click,2024-11-11,True,2024-11-11,2024-11


In [26]:
# Action count for sale and nosale days

agg_action_count_sale_nosale = (user_log_1
       .groupby(['user_id', 'merchant_id', 'is_sale_day', 'action'])
       .size()
       .reset_index(name='count'))

# Pivot table: sale_day vs non-sale & action counts
features_action_count_sale_nosale = agg_action_count_sale_nosale.pivot_table(
    index=['user_id', 'merchant_id'],
    columns=['is_sale_day', 'action'],
    values='count',
    fill_value=0
)

# Flatten column names
features_action_count_sale_nosale.columns = [
    f"{'sale' if sd else 'nosale'}_{act}"
    for sd, act in features_action_count_sale_nosale.columns
]

features_action_count_sale_nosale = features_action_count_sale_nosale.reset_index()


features_action_count_sale_nosale

,user_id,merchant_id,nosale_add-to-cart,nosale_click,nosale_favourite,nosale_purchase,sale_add-to-cart,sale_click,sale_favourite,sale_purchase
0,1,1019,0.0,0.0,0.0,0.0,0.0,10.0,0.0,4.0
1,1,4026,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0
2,1,4177,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,4335,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,4,434,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
2821933,424167,2857,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2821934,424167,3959,0.0,4.0,0.0,1.0,0.0,2.0,0.0,0.0
2821935,424170,525,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2821936,424170,3469,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
features_action_count_sale_nosale['sale_purchase_click'] = features_action_count_sale_nosale['sale_click'] / (features_action_count_sale_nosale['sale_purchase'] + 1)
features_action_count_sale_nosale['sale_purchase_fav'] = features_action_count_sale_nosale['sale_favourite'] / (features_action_count_sale_nosale['sale_purchase'] + 1)
features_action_count_sale_nosale['sale_purchase_add'] = features_action_count_sale_nosale['sale_add-to-cart'] / (features_action_count_sale_nosale['sale_purchase'] + 1)
features_action_count_sale_nosale['nosale_purchase_click'] = features_action_count_sale_nosale['nosale_click'] / (features_action_count_sale_nosale['nosale_purchase'] + 1)
features_action_count_sale_nosale['nosale_purchase_fav'] = features_action_count_sale_nosale['nosale_favourite'] / (features_action_count_sale_nosale['nosale_purchase'] + 1)
features_action_count_sale_nosale['nosale_purchase_add'] = features_action_count_sale_nosale['nosale_add-to-cart'] / (features_action_count_sale_nosale['nosale_purchase'] + 1)

In [29]:
features_action_count_sale_nosale.head(5)

,user_id,merchant_id,nosale_add-to-cart,nosale_click,nosale_favourite,nosale_purchase,sale_add-to-cart,sale_click,sale_favourite,sale_purchase,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add
0,1,1019,0.0,0.0,0.0,0.0,0.0,10.0,0.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0
1,1,4026,0.0,4.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
2,1,4177,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1,4335,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,434,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0


In [35]:
final_full_data.head()

,user_id,merchant_id,label,age,sex,marital_status,job_industry
0,393595.0,794.0,0.0,35.0,F,Divorced,Hospitality/Service
1,287580.0,2447.0,0.0,17.0,F,Married,Engineering/Construction
2,203618.0,598.0,0.0,26.0,F,Married,Engineering/Construction
3,143560.0,2030.0,0.0,30.0,F,Divorced,Others
4,324173.0,1875.0,0.0,32.0,F,Married,Others


In [ ]:
final_full_data_1 = final_full_data.copy()
final_full_data_1 = final_full_data_1.merge(features_action_count_sale_nosale[['user_id', 'merchant_id', 'sale_purchase_click', 'sale_purchase_fav', 'sale_purchase_add',
                                                  'nosale_purchase_click', 'nosale_purchase_fav', 'nosale_purchase_add']],
                                        on=['user_id','merchant_id'], how='left')
final_full_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272252 entries, 0 to 272251
Data columns (total 9 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                272252 non-null  float64
 1   merchant_id            272252 non-null  float64
 2   label                  272252 non-null  float64
 3   sale_purchase_click    268547 non-null  float64
 4   sale_purchase_fav      268547 non-null  float64
 5   sale_purchase_add      268547 non-null  float64
 6   nosale_purchase_click  268547 non-null  float64
 7   nosale_purchase_fav    268547 non-null  float64
 8   nosale_purchase_add    268547 non-null  float64
dtypes: float64(9)
memory usage: 18.7 MB


In [ ]:
final_full_data_1.head(5)

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0


In [ ]:
final_full_data_1[(final_full_data_1['sale_purchase_click'] > 0)]

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
272247,1925.0,3266.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0
272248,55363.0,4079.0,0.0,0.181818,2.0,2.0,0.0,0.0,0.0
272249,68457.0,742.0,0.0,0.333333,1.0,1.0,0.0,0.0,0.0
272250,15500.0,4589.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0


In [ ]:
df_purchase = user_log_2[user_log_2['action'] == 'purchase'].copy()
df_purchase.info()

In [ ]:
df_purchase['month'] = df_purchase['datetime'].dt.to_period('M')
df_purchase.head()

,user_id,item_id,merchant_id,action,datetime,is_sale_day,month
0,234512,240182,3018,purchase,2024-11-11,True,2024-11
1,234512,137298,3271,purchase,2024-11-11,True,2024-11
43,356311,1017725,2768,purchase,2024-06-04,False,2024-06
59,356311,575822,4650,purchase,2024-08-07,False,2024-08
63,356311,238223,4650,purchase,2024-08-07,False,2024-08


In [ ]:
monthly_purchase = df_purchase.groupby(['user_id', 'merchant_id', 'month', 'is_sale_day']).size().reset_index(name='count')
monthly_purchase.head()

,user_id,merchant_id,month,is_sale_day,count
0,1,1019,2024-11,True,4
1,1,4026,2024-10,False,1
2,4,1186,2024-11,True,1
3,6,587,2024-09,True,1
4,6,1356,2024-11,True,1


In [ ]:
# Pivot table: sale_day vs non-sale & action counts
features2 = monthly_purchase.pivot_table(
    index=['user_id', 'merchant_id'],
    columns=['is_sale_day', 'month'],
    values='count',
    fill_value=0
)

# Flatten column names
features2.columns = [
    f"{'sale' if sd else 'nosale'}_{act}"
    for sd, act in features2.columns
]

features2 = features2.reset_index()
features2.head()

,user_id,merchant_id,nosale_2024-06,nosale_2024-07,nosale_2024-08,nosale_2024-09,nosale_2024-10,nosale_2024-11,sale_2024-06,sale_2024-07,sale_2024-08,sale_2024-09,sale_2024-10,sale_2024-11
0,1,1019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
1,1,4026,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4,1186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,6,587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,6,1356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [ ]:
features2['purchase_ratio_06'] = features2['sale_2024-06'] / (features2['nosale_2024-06'] + 1)
features2['purchase_ratio_07'] = features2['sale_2024-07'] / (features2['nosale_2024-07'] + 1)
features2['purchase_ratio_08'] = features2['sale_2024-08'] / (features2['nosale_2024-08'] + 1)
features2['purchase_ratio_09'] = features2['sale_2024-09'] / (features2['nosale_2024-09'] + 1)
features2['purchase_ratio_10'] = features2['sale_2024-10'] / (features2['nosale_2024-10'] + 1)
features2['purchase_ratio_11'] = features2['sale_2024-11'] / (features2['nosale_2024-11'] + 1)
features2['purchase_ratio'] = (features2['purchase_ratio_06']
                               + features2['purchase_ratio_07']
                               + features2['purchase_ratio_08']
                               + features2['purchase_ratio_09']
                               + features2['purchase_ratio_10']
                               + features2['purchase_ratio_11']) / 6

In [ ]:
features2

,user_id,merchant_id,nosale_2024-06,nosale_2024-07,nosale_2024-08,nosale_2024-09,nosale_2024-10,nosale_2024-11,sale_2024-06,sale_2024-07,...,sale_2024-09,sale_2024-10,sale_2024-11,purchase_ratio_06,purchase_ratio_07,purchase_ratio_08,purchase_ratio_09,purchase_ratio_10,purchase_ratio_11,purchase_ratio
0,1,1019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.0,0.666667
1,1,4026,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,4,1186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667
3,6,587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.166667
4,6,1356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496675,424163,3826,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667
496676,424164,606,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.166667
496677,424167,1200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.333333
496678,424167,3959,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
final_full_data

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
272247,1925.0,3266.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0
272248,55363.0,4079.0,0.0,0.181818,2.0,2.0,0.0,0.0,0.0
272249,68457.0,742.0,0.0,0.333333,1.0,1.0,0.0,0.0,0.0
272250,15500.0,4589.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0


In [ ]:
final_full_data = final_full_data.merge(features2[['user_id', 'merchant_id', 'purchase_ratio']],
                                        on=['user_id','merchant_id'], how='left')
final_full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 272252 entries, 0 to 272251
Data columns (total 10 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   user_id                272252 non-null  float64
 1   merchant_id            272252 non-null  float64
 2   label                  272252 non-null  float64
 3   sale_purchase_click    268547 non-null  float64
 4   sale_purchase_fav      268547 non-null  float64
 5   sale_purchase_add      268547 non-null  float64
 6   nosale_purchase_click  268547 non-null  float64
 7   nosale_purchase_fav    268547 non-null  float64
 8   nosale_purchase_add    268547 non-null  float64
 9   purchase_ratio         268547 non-null  float64
dtypes: float64(10)
memory usage: 20.8 MB


In [ ]:
final_full_data.head(20)

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667
5,141726.0,1272.0,0.0,0.090909,2.0,2.0,0.0,0.0,0.0,0.333333
6,117868.0,415.0,0.0,0.500000,2.0,2.0,0.0,0.0,0.0,0.333333
7,170952.0,212.0,0.0,1.000000,1.0,1.0,0.0,0.0,0.0,0.166667
8,186785.0,3173.0,0.0,3.000000,3.0,3.0,0.0,0.0,0.0,0.500000
9,96875.0,4048.0,0.0,0.200000,1.0,1.0,0.0,0.0,0.0,0.166667


In [ ]:
df1 = user_log_2.copy()
df1['month'] = df1['datetime'].dt.to_period('M')
df1['date'] = df1['datetime'].dt.date
df1

,user_id,item_id,merchant_id,action,datetime,is_sale_day,month,date
0,234512,240182,3018,purchase,2024-11-11,True,2024-11,2024-11-11
1,234512,137298,3271,purchase,2024-11-11,True,2024-11,2024-11-11
2,234512,137298,3271,click,2024-11-11,True,2024-11,2024-11-11
3,234512,719701,4655,click,2024-11-11,True,2024-11,2024-11-11
4,234512,137298,3271,click,2024-11-11,True,2024-11,2024-11-11
...,...,...,...,...,...,...,...,...
10894811,208016,466472,1346,click,2024-11-10,False,2024-11,2024-11-10
10894812,208016,107662,1346,click,2024-11-10,False,2024-11,2024-11-10
10894813,208016,1058313,1346,click,2024-11-10,False,2024-11,2024-11-10
10894814,208016,634856,1346,click,2024-11-10,False,2024-11,2024-11-10


In [ ]:
monthly_stats = df1.groupby(['user_id', 'merchant_id', 'month']).agg(
    total_actions=('action', 'count'),
    active_days=('date', lambda x: x.nunique())
).reset_index()

monthly_stats['actions_days'] = monthly_stats['total_actions'] / monthly_stats['active_days']
monthly_stats

,user_id,merchant_id,month,total_actions,active_days,actions_days
0,1,1019,2024-11,14,1,14.0
1,1,4026,2024-10,5,2,2.5
2,1,4177,2024-10,1,1,1.0
3,1,4335,2024-11,1,1,1.0
4,4,434,2024-10,1,1,1.0
...,...,...,...,...,...,...
3042386,424167,2857,2024-11,1,1,1.0
3042387,424167,3959,2024-11,7,2,3.5
3042388,424170,525,2024-11,2,1,2.0
3042389,424170,3469,2024-11,1,1,1.0


In [ ]:
months = pd.period_range('2024-06', '2024-11', freq='M')
all_pairs = (
    monthly_stats[['user_id', 'merchant_id']]
    .drop_duplicates()
    .assign(key=1)
)
# all_pairs

month_df = (
    pd.DataFrame({'month': months, 'key': 1})
)

full_index = (
    all_pairs.merge(month_df, on='key')
    .drop(columns='key')
)
# full_index

full_month_stats = (
    full_index.merge(monthly_stats, on=['user_id', 'merchant_id', 'month'], how='left')
    .fillna({'total_actions': 0, 'active_days': 0, 'actions_days': 0})
)


In [ ]:
fina_full_month_stats_1 = full_month_stats.groupby(['user_id', 'merchant_id'])['actions_days'].mean().reset_index(name='avg_actions_per_day_6m')

In [ ]:
fina_full_month_stats_2 = full_month_stats.groupby(['user_id', 'merchant_id'])['active_days'].mean().reset_index(name='avg_active_days_6m')

In [ ]:
final_full_data

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667
...,...,...,...,...,...,...,...,...,...,...
272247,1925.0,3266.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667
272248,55363.0,4079.0,0.0,0.181818,2.0,2.0,0.0,0.0,0.0,0.333333
272249,68457.0,742.0,0.0,0.333333,1.0,1.0,0.0,0.0,0.0,0.166667
272250,15500.0,4589.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667


In [ ]:
final_full_data = final_full_data.merge(fina_full_month_stats_1[['user_id', 'merchant_id', 'avg_actions_per_day_6m']], on=['user_id','merchant_id'], how='left')
final_full_data = final_full_data.merge(fina_full_month_stats_2[['user_id', 'merchant_id', 'avg_active_days_6m']], on=['user_id','merchant_id'], how='left')
final_full_data

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio,avg_actions_per_day_6m,avg_active_days_6m
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333,0.966667,1.000000
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667,1.000000,0.166667
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333,1.333333,0.166667
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667,6.166667,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...
272247,1925.0,3266.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667
272248,55363.0,4079.0,0.0,0.181818,2.0,2.0,0.0,0.0,0.0,0.333333,2.166667,0.333333
272249,68457.0,742.0,0.0,0.333333,1.0,1.0,0.0,0.0,0.0,0.166667,0.500000,0.166667
272250,15500.0,4589.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667


In [ ]:
df2 = user_log_2.copy()
df2['datetime'] = pd.to_datetime(df2['datetime'])
df2['date'] = df2['datetime'].dt.normalize()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10894816 entries, 0 to 10894815
Data columns (total 7 columns):
 #   Column       Dtype         
---  ------       -----         
 0   user_id      int64         
 1   item_id      int64         
 2   merchant_id  int64         
 3   action       object        
 4   datetime     datetime64[ns]
 5   is_sale_day  bool          
 6   date         datetime64[ns]
dtypes: bool(1), datetime64[ns](2), int64(3), object(1)
memory usage: 509.1+ MB


In [ ]:
# Sort theo user_id, merchant_id và datetime
df2 = df2.sort_values(['user_id', 'merchant_id', 'datetime'])

# Tính chênh lệch ngày giữa các hoạt động
df2['activity_gap'] = df2.groupby(['user_id', 'merchant_id'])['date'].diff().dt.days

global_max_date = df2['date'].max()


In [ ]:
# 4. Aggregate: lấy max internal gap và last_date cho mỗi cặp
agg = (
    df2.groupby(['user_id','merchant_id'])
       .agg(
           max_internal_gap = ('activity_gap', 'max'),  # có thể NaN nếu chỉ 1 log
           last_date = ('date', 'max')
       )
       .reset_index()
)

# 5. Chuyển NaN max_internal_gap -> 0 (để dễ so sánh)
agg['max_internal_gap'] = agg['max_internal_gap'].fillna(0)

# 6. Tính last_gap (khoảng từ last_date tới global_max_date)
agg['last_gap'] = (global_max_date - agg['last_date']).dt.days

# 7. Lấy max giữa max_internal_gap và last_gap
agg['longest_inactivity_days'] = agg[['max_internal_gap','last_gap']].max(axis=1).astype(int)

# 8. (Tuỳ chọn) giữ các cột cần thiết
longest_gap = agg[['user_id','merchant_id','longest_inactivity_days']]


In [ ]:
longest_gap.head(5)

,user_id,merchant_id,longest_inactivity_days
0,1,1019,1
1,1,4026,22
2,1,4177,25
3,1,4335,1
4,4,434,11


In [ ]:
final_full_data.head(5)

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio,avg_actions_per_day_6m,avg_active_days_6m
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333,0.966667,1.000000
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667,1.000000,0.166667
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333,1.333333,0.166667
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667,6.166667,0.166667


In [ ]:
final_full_data = final_full_data.merge(longest_gap[['user_id', 'merchant_id', 'longest_inactivity_days']], on=['user_id','merchant_id'], how='left')
final_full_data.head(5)

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio,avg_actions_per_day_6m,avg_active_days_6m,longest_inactivity_days_x,longest_inactivity_days_y
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333,0.966667,1.000000,6.0,6.0
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667,1.000000,0.166667,0.0,1.0
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333,1.333333,0.166667,0.0,1.0
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667,0.0,1.0
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667,6.166667,0.166667,0.0,1.0


In [ ]:
# Lọc hành động mua
df_purchase_new = df2[df2['action'] == 'purchase'].copy()

# Sort theo user_id, merchant_id và datetime
df_purchase_new = df_purchase_new.sort_values(['user_id', 'merchant_id', 'datetime'])

# Tính chênh lệch ngày giữa các hoạt động
df_purchase_new['purchase_gap'] = df_purchase_new.groupby(['user_id', 'merchant_id'])['date'].diff().dt.days

global_max_date = df2['date'].max()

# 4. Aggregate: lấy max internal gap và last_date cho mỗi cặp
agg = (
    df_purchase_new.groupby(['user_id','merchant_id'])
       .agg(
           max_internal_gap = ('purchase_gap', 'max'),  # có thể NaN nếu chỉ 1 log
           last_date = ('date', 'max')
       )
       .reset_index()
)

# 5. Chuyển NaN max_internal_gap -> 0 (để dễ so sánh)
agg['max_internal_gap'] = agg['max_internal_gap'].fillna(0)

# 6. Tính last_gap (khoảng từ last_date tới global_max_date)
agg['last_gap'] = (global_max_date - agg['last_date']).dt.days

# 7. Lấy max giữa max_internal_gap và last_gap
agg['longest_purchase_gap'] = agg[['max_internal_gap','last_gap']].max(axis=1).astype(int)

# 8. (Tuỳ chọn) giữ các cột cần thiết
longest_purchase_gap = agg[['user_id','merchant_id','longest_purchase_gap']]


In [ ]:
final_full_data = final_full_data.merge(longest_purchase_gap[['user_id', 'merchant_id', 'longest_purchase_gap']], on=['user_id','merchant_id'], how='left')
final_full_data.head(5)

,user_id,merchant_id,label,sale_purchase_click,sale_purchase_fav,sale_purchase_add,nosale_purchase_click,nosale_purchase_fav,nosale_purchase_add,purchase_ratio,avg_actions_per_day_6m,avg_active_days_6m,longest_inactivity_days_x,longest_inactivity_days_y,longest_purchase_gap
0,393595.0,794.0,0.0,2.000000,2.0,2.0,0.0,0.0,0.0,0.333333,0.966667,1.000000,6.0,6.0,1.0
1,287580.0,2447.0,0.0,0.166667,1.0,1.0,0.0,0.0,0.0,0.166667,1.000000,0.166667,0.0,1.0,1.0
2,203618.0,598.0,0.0,0.285714,2.0,2.0,0.0,0.0,0.0,0.333333,1.333333,0.166667,0.0,1.0,1.0
3,143560.0,2030.0,0.0,0.500000,1.0,1.0,0.0,0.0,0.0,0.166667,0.333333,0.166667,0.0,1.0,1.0
4,324173.0,1875.0,0.0,0.027027,1.0,1.0,0.0,0.0,0.0,0.166667,6.166667,0.166667,0.0,1.0,1.0
